In [1]:
import os
import pandas as pd
import pandas as pd
from tqdm.auto import tqdm
from collections import defaultdict
data=defaultdict(lambda: defaultdict(list))

In [2]:
v17=pd.read_excel(open('result_Human_eva_100ch.xlsx', 'rb'),sheet_name='Model3')
seallm=pd.read_excel(open('result_Human_eva_100ch.xlsx', 'rb'),sheet_name='Model2')
openthaigpt=pd.read_excel(open('result_Human_eva_100ch.xlsx', 'rb'),sheet_name='Model1')
file="typhoon_polm_and_openthai_13b.xlsx"
typhoon=pd.read_excel(open(file, 'rb'),sheet_name='typhoon-instruct-0130')
file="polm_and_openthai_13b.xlsx"
polylm=pd.read_excel(open(file, 'rb'),sheet_name='7b_xquad_polylmChat_ins_0shot_m')
openthaigpt13b=pd.read_excel(open(file, 'rb'),sheet_name='7b_xquad_openthai13b_ins_0shot_')

In [3]:
from closeai import openai # After providing your explanation, 

In [4]:
from config import system_prompt,model_name

In [5]:
print(system_prompt)

Please evaluate these answers based on their accuracy and relevance to the provided passage that based on the Criteria:
1. The Answer is Correct concerning the Reference Answer. Do you agree or disagree?
Determine if the given answer accurately matches the reference answer provided. The correctness here means the answer must directly correspond to the reference answer, ensuring factual accuracy.
2. The Answer Includes Relevant, Additional Information from the Context. Do you agree or disagree?
Assess whether the answer provides extra details that are not only correct but also relevant and enhance the understanding of the topic as per the information given in the context.
3. The Answer Includes Additional, Irrelevant Information from the Context. Do you agree or disagree?
Check if the answer contains extra details that, while related to the context, do not directly pertain to the question asked. This information is not necessary for answering the question and is considered a digression.

In [6]:
print(model_name)

gpt-4


In [7]:
from config import get_bool,score

In [8]:
get_bool("""1. Is the Answer Correct concerning the Reference Answer?
Yes, the prediction answer is correct as it matches the reference answer.

2. Does the Answer Include Relevant, Additional Information from the Passage?
Yes, the prediction answer includes relevant additional information from the passage by mentioning the names of some of the examination boards (CBSE, CISCE, NENBSE).

3. Does the Answer Include Additional, Irrelevant Information from the Passage?
No, the prediction answer does not include any irrelevant information from the passage.

4. Does the Answer Include Information Not Found in the Passage?
No, the prediction answer does not include any information that is not found in the passage.""")

In [9]:
def get_ans(context, question,reference_answer, prediction_answer,show=False):
    user_prompt=f'''Passage: {context}
Question: {question}
Reference Answer: "{reference_answer}"
Prediction Answer: "{prediction_answer}"
'''
    if show:
        print(user_prompt)
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        n=1,
        temperature=0.2,
        # top_p=1,
        # # frequency_penalty=0.0,
        # presence_penalty=1,
        max_tokens=1024,
    )
    return response['choices'][0]['message']['content']

In [10]:
def recheck(context, question,reference_answer, prediction_answer,show=False):
    ok4=False
    while ok4==False:
        try:
            output=get_ans(context, question,reference_answer, prediction_answer,show=show)
            q1,q2,q3,q4=get_bool(output)
            ok4=True
        except:
            ok4=False
    return output

In [11]:
print(recheck(
    v17["context"][0],
    v17["question"][0],
    v17["references"][0],
    v17["predictions"][0],
))

1. The Answer is Correct concerning the Reference Answer. Do you agree or disagree?
Agree
2. The Answer Includes Relevant, Additional Information from the Context. Do you agree or disagree?
Disagree
3. The Answer Includes Additional, Irrelevant Information from the Context. Do you agree or disagree?
Disagree
4. The Answer Includes Information Not Found in the Context. Do you agree or disagree?
Disagree


In [12]:
get_bool("""1. The Answer is Correct concerning the Reference Answer. Do you agree or disagree?
Agree
2. The Answer Includes Relevant, Additional Information from the Context. Do you agree or disagree?
Disagree
3. The Answer Includes Additional, Irrelevant Information from the Context. Do you agree or disagree?
Disagree
4. The Answer Includes Information Not Found in the Context. Do you agree or disagree?
Disagree""")

[True, False, False, False]

In [13]:
first_10_data=defaultdict(list)

In [14]:
print(system_prompt)

Please evaluate these answers based on their accuracy and relevance to the provided passage that based on the Criteria:
1. The Answer is Correct concerning the Reference Answer. Do you agree or disagree?
Determine if the given answer accurately matches the reference answer provided. The correctness here means the answer must directly correspond to the reference answer, ensuring factual accuracy.
2. The Answer Includes Relevant, Additional Information from the Context. Do you agree or disagree?
Assess whether the answer provides extra details that are not only correct but also relevant and enhance the understanding of the topic as per the information given in the context.
3. The Answer Includes Additional, Irrelevant Information from the Context. Do you agree or disagree?
Check if the answer contains extra details that, while related to the context, do not directly pertain to the question asked. This information is not necessary for answering the question and is considered a digression.

In [15]:
from tqdm.auto import tqdm

# first10_v17_zeros_df = v17_zeros_df.head(50)
# first10_openthai_zeros_df = openthai_zeros_df.head(50)
# first10_seallm_zeros_df = seallm_zeros_df.head(50)
for i in tqdm(list(range(len(v17)))):
    j=v17
    first_10_data["v17"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))
    j=openthaigpt
    first_10_data["openthaigpt"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))
    j=seallm
    first_10_data["seallm"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))
    j=polylm
    first_10_data["polylm"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))
    j=openthaigpt13b
    first_10_data["openthaigpt13b"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))
    j=typhoon
    first_10_data["typhoon"].append(recheck(j["context"][i],j["question"][i],j["references"][i],j["predictions"][i]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
first_10_data.keys()

dict_keys(['v17', 'openthaigpt', 'seallm', 'polylm', 'openthaigpt13b', 'typhoon'])

In [17]:
raw_data_df=pd.DataFrame.from_dict({
    "context":v17["context"].to_list(),
    "question":v17["question"].to_list(),
    "references":v17["references"].to_list(),
    # "v17":v17["predictions"].to_list(),
    # "v17_gpt4":first_10_data["v17"],
    # # "v17_bool":[get_bool(i) for i in first_10_data["v17_zeros_df"]],
    # "v17_score":[check_score_inpaper(get_bool(i)) for i in first_10_data["v17_zeros_df"]],
    # "openthaigpt":first10_openthai_zeros_df["predictions"].to_list()[50:100],
    # "openthaigpt_gpt4":first_10_data["openthai_zeros_df"],
    # # "openthaigpt_bool":[get_bool(i) for i in first_10_data["openthai_zeros_df"]],
    # "openthaigpt_score":[check_score_inpaper(get_bool(i)) for i in tqdm(first_10_data["openthai_zeros_df"])],
    # "seallm":first10_seallm_zeros_df["predictions"].to_list()[50:100],
    # "seallm_gpt4":first_10_data["seallm_zeros_df"],
    # # "seallm_bool":[get_bool(i) for i in first_10_data["seallm_zeros_df"]],
    # "seallm_score":[check_score_inpaper(get_bool(i)) for i in first_10_data["seallm_zeros_df"]],
})



In [18]:
for model in list(first_10_data.keys()):
    l= [get_bool(i) for i in first_10_data[model]]
    temp_df=None
    if model == "v17":
        raw_data_df["v17"]=v17["predictions"].to_list()
        raw_data_df["v17_gpt4"]=first_10_data["v17"]
        temp_df=v17
    elif model == "openthaigpt":
        raw_data_df["openthaigpt"]=openthaigpt["predictions"].to_list()
        raw_data_df["openthaigpt_gpt4"]=first_10_data["openthaigpt"]
        temp_df=openthaigpt
    elif model == "seallm":
        raw_data_df["seallm"]=seallm["predictions"].to_list()
        raw_data_df["seallm_gpt4"]=first_10_data["seallm"]
        temp_df=seallm
    elif model == "polylm":
        raw_data_df["polylm"]=polylm["predictions"].to_list()
        raw_data_df["polylm_gpt4"]=first_10_data["polylm"]
        temp_df=polylm
    elif model == "openthaigpt13b":
        raw_data_df["openthaigpt13b"]=openthaigpt13b["predictions"].to_list()
        raw_data_df["openthaigpt13b_gpt4"]=first_10_data["openthaigpt13b"]
        temp_df=openthaigpt13b
    elif model == "typhoon":
        raw_data_df["typhoon"]=typhoon["predictions"].to_list()
        raw_data_df["typhoon_gpt4"]=first_10_data["typhoon"]
        temp_df=typhoon
    _temp = {"q"+str(i+1):[] for i in range(0,4)}
    temp_bool=[get_bool(i) for i in first_10_data[model]]
    for i in tqdm(temp_bool):
        if i == None:
            print(model)
        _temp["q1"].append(int(bool(i[0])))
        _temp["q2"].append(int(bool(i[1])))
        _temp["q3"].append(int(bool(i[2])))
        _temp["q4"].append(int(bool(i[3])))
    raw_data_df[model+"_q1"] = _temp["q1"]
    raw_data_df[model+"_q2"] = _temp["q2"]
    raw_data_df[model+"_q3"] = _temp["q3"]
    raw_data_df[model+"_q4"] = _temp["q4"]
    # for q in l:
    #     model_name = model.replace("_zeros_df","")
    #     for index,ans in enumerate(q):
    #         raw_data_df[model_name+"_q"+str(index+1)] = int(ans)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [19]:
raw_data_df.to_csv("gpt4-100.csv",index=False)
raw_data_df.to_excel("gpt4-100.xlsx",index=False)

In [20]:
for i in list(first_10_data.keys()):
    print(i)
    print("q1: "+str(raw_data_df[f"{i}_q1"].sum()))
    print("q2: "+str(raw_data_df[f"{i}_q2"].sum()))
    print("q3: "+str(raw_data_df[f"{i}_q3"].sum()))
    print("q4: "+str(raw_data_df[f"{i}_q4"].sum()))
    print("---------------")

v17
q1: 64
q2: 8
q3: 28
q4: 5
---------------
openthaigpt
q1: 58
q2: 13
q3: 28
q4: 31
---------------
seallm
q1: 76
q2: 48
q3: 32
q4: 31
---------------
polylm
q1: 73
q2: 17
q3: 20
q4: 5
---------------
openthaigpt13b
q1: 61
q2: 25
q3: 34
q4: 38
---------------
typhoon
q1: 76
q2: 31
q3: 25
q4: 25
---------------


In [21]:
from pythainlp.tokenize import word_tokenize
from collections import defaultdict

In [22]:
count_words=defaultdict(list)

In [23]:
for i in list(first_10_data.keys()):
    for j in raw_data_df[i].tolist():
        count_words[i].append(len(word_tokenize(j,keep_whitespace=False)))
    print(str(i)+"\t"+str(sum(count_words[i])/len(count_words[i])))

v17	5.5
openthaigpt	10.35
seallm	27.81
polylm	11.96
openthaigpt13b	17.08
typhoon	18.33
